In [ ]:
import pymongo
import pandas as pd
from tqdm import tqdm

from notebook_modules.database import Database
from notebook_modules.half import Half
from notebook_modules.lists import make_list
from notebook_modules.plots import make_distribution_plot

In [ ]:
db = Database()
assert db.client, "No database client available!"
stackoverflow = db.client["stackoverflow"]
tags = stackoverflow["tags"]
posts = stackoverflow["posts"]

In [ ]:
halves = Half.make_halves(2019, 2020)

In [ ]:
def aggregate_full(timespan):
    cursor = posts.aggregate([
        {"$match": {
            "PostTypeId": 1,
            "CreationDate": {
                #"$gte": timespan.start,
                "$lt": timespan.end
            }
        }},
        {"$unwind": "$Tags"},
        {"$group": {
            "_id": "$Tags",
            "frequency": {"$sum": 1}
        }},
        {"$sort": {
            "frequency": -1
        }}
    ])
    result = list(cursor)
    if len(result) == 0:
        return pd.DataFrame()
    else:
        df = pd.DataFrame(result)
        df.rename({"_id": "tag"}, axis="columns", inplace=True)
        return df

In [ ]:
def distribute(aggr):
    dist = []
    unique_tags = len(aggr.index)
    total_tags = aggr.frequency.sum()
    resolution = 100
    for p in range(0, resolution + 1):
        percentage = p / resolution
        index = int(unique_tags * percentage)
        part = aggr.head(index)
        share = part.frequency.sum() / total_tags
        dist.append([percentage, share, len(part.index)])
    return pd.DataFrame(dist, columns=["percentage", "share", "count"])

In [ ]:
for half in tqdm(halves, unit="half", ascii=True):
    tags_from_posts = aggregate_full(half)
    if tags_from_posts.empty:
        dist = pd.DataFrame(columns=["percentage", "share", "count"])
    else:
        dist = distribute(tags_from_posts)
    make_list("distribution", str(half), dist)
    make_distribution_plot("distribution", str(half), dist)